In [74]:
pip install yfinance riskfolio-lib


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [75]:
import yfinance as yf
import pandas as pd
import riskfolio as rp
import matplotlib.pyplot as plt

In [76]:
# Fetch stock data
tickers = ['NVDA', 'SMCI', 'KNSL', 'GCT', 'PDD', 'CRWD', 'AVGO', 'SNOW']
start_date = '2021-01-01'
end_date = '2023-03-20'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Calculate returns
returns = data.pct_change().dropna()

# Create a portfolio object
port = rp.Portfolio(returns)

# Estimate expected returns and covariance matrix
port.assets_stats(method_mu='hist', method_cov='hist')

# Define the asset bounds and constraints
port.ainequality = None
port.binequality = None
port.lowerbound = 0.05
port.upperbound = 0.5
port.budget = 1

# Optimize the portfolio using ERP
model = 'Classic' # Could be 'Classic' (historical), 'BL' (Black Litterman), or 'FM' (Factor Model)
rm = 'MV' # Risk measure could be 'MV' (variance), 'MAD' (mean absolute deviation), or 'MSV' (semi-variance)
rf = 0.0 # Risk-free rate
b = None # Risk budgeting vector (None for ERP)

w = port.rp_optimization(model=model, rm=rm, rf=rf, b=b)

# Print the optimal weights
print(w.T)

[*********************100%%**********************]  8 of 8 completed

            AVGO     CRWD     GCT     KNSL    NVDA      PDD     SMCI    SNOW
weights 17.6368% 12.9274% 4.3144% 22.5236% 9.9293% 11.1851% 11.7565% 9.7268%


In [77]:
# Print the optimal weights
print("Optimal Weights:")
print(w.T)

# Calculate and display correlation matrix
print("\nCorrelation Matrix:")
corr_matrix = port.returns.corr()
print(corr_matrix)

Optimal Weights:
            AVGO     CRWD     GCT     KNSL    NVDA      PDD     SMCI    SNOW
weights 17.6368% 12.9274% 4.3144% 22.5236% 9.9293% 11.1851% 11.7565% 9.7268%

Correlation Matrix:
Ticker      AVGO      CRWD       GCT      KNSL      NVDA       PDD      SMCI   
Ticker                                                                         
AVGO   100.0000%  42.3773%  -2.3085%  27.6516%  73.5578%  34.7522%  41.4915%  \
CRWD    42.3773% 100.0000%  -6.8972%   3.3480%  53.5104%  20.8041%  36.4657%   
GCT     -2.3085%  -6.8972% 100.0000%   2.3832%  -8.8998%   1.9583%  -4.1565%   
KNSL    27.6516%   3.3480%   2.3832% 100.0000%  18.6734%   3.8102%   7.8069%   
NVDA    73.5578%  53.5104%  -8.8998%  18.6734% 100.0000%  30.4421%  47.6507%   
PDD     34.7522%  20.8041%   1.9583%   3.8102%  30.4421% 100.0000%  26.3039%   
SMCI    41.4915%  36.4657%  -4.1565%   7.8069%  47.6507%  26.3039% 100.0000%   
SNOW    48.3765%  62.8635% -10.8659%  -4.7378%  55.8036%  25.5328%  38.5533%   

Ticker 

In [78]:
# Calculate volatilities
daily_volatility = returns.std()
monthly_volatility = returns.resample('M').sum().std()
annual_volatility = returns.resample('Y').sum().std()

# Combine volatilities into a DataFrame
volatilities = pd.DataFrame({
    'Daily Volatility': daily_volatility,
    'Monthly Volatility': monthly_volatility,
    'Annual Volatility': annual_volatility
})

# Format volatilities as percentages
volatilities = volatilities.applymap(lambda x: f"{x:.2%}")

# Display the volatilities as a plain text table
print("Ticker | Daily Volatility | Monthly Volatility | Annual Volatility")
print("-------|------------------|--------------------|-----------------")
for ticker, row in volatilities.iterrows():
    print(f"{ticker:6} | {row['Daily Volatility']:>17} | {row['Monthly Volatility']:>18} | {row['Annual Volatility']:>17}")

Ticker | Daily Volatility | Monthly Volatility | Annual Volatility
-------|------------------|--------------------|-----------------
AVGO   |             2.13% |              9.14% |             6.07%
CRWD   |             3.48% |             13.24% |            57.71%
GCT    |            19.42% |             53.01% |            34.37%
KNSL   |             2.95% |             13.33% |             8.53%
NVDA   |             3.84% |             20.63% |            55.48%
PDD    |             4.71% |             22.88% |            37.51%
SMCI   |             3.96% |             19.81% |             0.40%
SNOW   |             4.69% |              8.86% |             0.29%
